## Processing Corporate Actions as input transactions

In this notebook, we will show how you can book Corporate Action transactions into LUSID using LUSID's standard input transactions. For the purposes of this demo, we will show:

* A dividend payment in cash
* A dividend reinvestment in stock

### Setup LUSID

In [1]:
import os

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import format_transactions_response
from lusidtools.cocoon.transaction_type_upload import (
    create_transaction_type_configuration,
)

# Import data wrangling packages
import pandas as pd
import numpy as np
import json

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

### Load a sample portfolio

In this section we seed a new sample portfolio with 10 FTSE100 stocks and GBP cash. This portfolio will be used to demonstrate the corporate actions.

In [2]:
# Create a new scope

scope = create_scope_id().replace("-", "")
portfolio_code = "EQUITY_UK"

(For more information on scopes, please see the [scopes](https://support.finbourne.com/what-is-a-scope-in-lusid-and-how-is-it-used) documenation on the support page)

In [3]:
# Load a file of equity transactions

transactions_file = r"data/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

In [4]:
# The seed_data() function takes a file of transaction data
# and loads portfolios, instruments, and transactions into LUSID
# We use this function as a quick way of generating a demo portfolio

create_demo_portfolio = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
)

### Build a Transaction Portfolios API

We build a Transactions Portfolios API object so we can interact with the Transaction Portfolio methods. See the [API documentation](https://www.lusid.com/docs/api/#tag/Transaction-Portfolios) for a list of methods. 

In [5]:
# Create a transaction portfolios API

transactions_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

In [6]:
# View the holdings for today

response = transactions_portfolios_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(response, rename_properties=True)

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,38566eb9ff91f0/EQUITY_UK,P,132000.0,132000.0,660000.0,GBP,0.0,GBP
1,LUID_7XM08GZF,{},BHP,38566eb9ff91f0/EQUITY_UK,P,120000.0,120000.0,2160000.0,GBP,0.0,GBP
2,LUID_STGB38I6,{},Barclays,38566eb9ff91f0/EQUITY_UK,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
3,LUID_PVOJGULG,{},BP,38566eb9ff91f0/EQUITY_UK,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
4,LUID_Z57YKL4W,{},HSBC,38566eb9ff91f0/EQUITY_UK,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
5,LUID_RNHEK2PL,{},Morrisons,38566eb9ff91f0/EQUITY_UK,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
6,LUID_99M6G8U7,{},Tesco,38566eb9ff91f0/EQUITY_UK,P,200000.0,200000.0,400000.0,GBP,0.0,GBP
7,LUID_6JES517Q,{},Rightmove,38566eb9ff91f0/EQUITY_UK,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
8,LUID_7KLNIUU7,{},vodafone,38566eb9ff91f0/EQUITY_UK,P,900000.0,900000.0,900000.0,GBP,0.0,GBP
9,LUID_NE84MHW9,{},Anglo American plc,38566eb9ff91f0/EQUITY_UK,P,70000.0,70000.0,1400000.0,GBP,0.0,GBP


### Create the corporate actions transaction types

We use the transaction type configuration to tell LUSID how to interpret the economic impact of a transaction when building holdings. In our example:

* The transaction type of "DividendCash" should increase the cash in a portfolio
* The transaction type of "DividendReinvest" should increase the units/stock of an equity position 

In [7]:
# Create a transaction type for cash dividends

create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="DividendCash",
        description="A cash payments from dividends",
        transaction_class="custodian",
        transaction_group="custodian",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashCommitment",
            side="Side2",
            direction=1,
            properties=None,
            mappings=None,
        )
    ],
)

# Create a transaction type for stock reinvestment

create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="DividendReinvest",
        description="A dividend reinvestment",
        transaction_class="custodian",
        transaction_group="custodian",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=1,
            properties=None,
            mappings=None,
        )
    ],
)

print("Transaction Type upload completed")

ERROR:root:The following alias already exists: Type of DividendCash with source custodian
ERROR:root:The following alias already exists: Type of DividendReinvest with source custodian


Transaction Type upload completed


### Post the Corporate Action input transactions

For the Corporate Action, we have:

* A reinvestment of 10,000 Aviva units
* A £200,000 GBP payment from the BHP cash dividend 

These transactions are posted with a payment date of 1 March 2020.

In [8]:
corp_act_file = r"data/corpact_input_transactions.csv"
corp_act_df = pd.read_csv(corp_act_file)
corp_act_df["portfolio_code"] = portfolio_code

In [9]:
corp_act_df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,corpact_0001,DividendReinvest,01/03/2020,01/03/2020,10000,0,0,GBP,ftse_tracker,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,dividend,GBP,BHP,corpact_0002,DividendCash,01/03/2020,01/03/2020,200000,1,20000,GBP,ftse_tracker,GBP


In [10]:
mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Currency": "cash_transactions",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {"source": "$custodian"},
}

In [11]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=corp_act_df,
    mapping_required=mapping["required"],
    mapping_optional=mapping["optional"],
    file_type="transactions",
    identifier_mapping=mapping["identifier_mapping"],
    property_columns=[],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
print(f"number of successful portfolios requests: {len(succ)}")
print(f"number of failed portfolios requests    : {len(failed)}")

number of successful portfolios requests: 1
number of failed portfolios requests    : 0


### Holdings in Aviva and GBP cash on 31 January 2020 (before the Corporate Action)

* Portfolio has 132,000 units of Aviva
* Portfolio has £2,960,000 GBP cash

In [12]:
response = transactions_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-01-31",
)

holdings_df = lusid_response_to_data_frame(response, rename_properties=True)
holdings_df[holdings_df["Name(default-Properties)"].isin(["Aviva", "CCY_GBP"])]

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,38566eb9ff91f0/EQUITY_UK,P,132000.0,132000.0,660000.0,GBP,0.0,GBP
10,CCY_GBP,{},CCY_GBP,38566eb9ff91f0/EQUITY_UK,B,2960000.0,2960000.0,2960000.0,GBP,0.0,GBP


### Holdings in Aviva and GBP cash on 30 April 2020 (after the Corporate Action)

* Portfolio has 132,000 units of Aviva (+10k units)
* Portfolio has £3,160,000 GBP cash (+ £200,000 cash)

In [13]:
response = transactions_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-04-30",
)

holdings_df = lusid_response_to_data_frame(response, rename_properties=True)
holdings_df[holdings_df["Name(default-Properties)"].isin(["Aviva", "CCY_GBP"])]

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,38566eb9ff91f0/EQUITY_UK,P,142000.0,142000.0,660000.0,GBP,0.0,GBP
10,CCY_GBP,{},CCY_GBP,38566eb9ff91f0/EQUITY_UK,B,2980000.0,2980000.0,2980000.0,GBP,0.0,GBP
